# Serial Communication for flashing SW - STM32 board flashing via UART

Protocol Sample (from Computer's standpoint)

**Start of communication:**
1. MCU:     0x80 FF FF FF -> Receives a 0x80 in the MSB, indicating that a MCU is present in the bootloader phase. Bytes 1,2,3 (LSB) don't care.
2. PC:      0xC0 xx xx xx -> Answers with a 0xC0 in the MSB (0x80 + 0x40), indicating that a serial communication is available and new SW might be flashed. The number of 4-byte SW packets is provided in bytes 1,2,3.
3. MCU:     0x81 xx FF FF -> MCU starts to erase each memory sector. The sector being currently erased is displayed through bit 1.
4. MCU:     0x82 xx xx xx -> MCU confirms that it can receive the provided packets (0x81). It'll also replicate the number of expected packets to receive in the LSB.

---
**Binary data streaming (loop - normal operation):**
1. PC:      0xyy yy yy yy -> 4-byte package containing the SW binary.
2. MCU:     0x83 xx FF FF -> MCU confirms that the given sector in 'byte 1' is flashed.

**Binary data streaming (loop - error operation):**
1. PC:      0xyy yy yy yy -> 4-byte package containing the SW binary.
2. MCU:     0x8F 01 FF FF -> Error - MCU couldn't flash the last packet provided.

---
**End of transmission**
1. MCU:     0x84 01 FF FF ->MCU informs transmission has been completed succesfully according to the
expected number of bytes.

### Configure Serial Communication

In [1]:
# ********** Configure Serial Comm **********

import serial
class SerialInterface():
    
    def __init__(self):
        SerialInterface.serialPort = serial.Serial(port="COM3",
                                            baudrate=115200, 
                                            bytesize=8, 
                                            timeout=0.001,
                                            stopbits=serial.STOPBITS_ONE)

    # Read 8-bytes packge from serial buffer
    def readSerialBuffer(self, echo=1):
        if SerialInterface.serialPort.in_waiting > 0:
            serialString = SerialInterface.serialPort.read(4) # Read a 4-bytes stream
            SerialInterface.serialPort.reset_input_buffer() # Ignore all other data
            if echo == 1: print("     RX-> " , serialString.hex())
            return serialString
        else:
            return None
        
    # Write a hex string (ex.: "C0C1") to serial buffer
    def writeSerialBuffer(self,txBufferStr,echo=1):
        txBuffer = bytes.fromhex(txBufferStr)
        SerialInterface.serialPort.write(txBuffer)
        if echo == 1: print("     TX-> ", txBufferStr)

    # Close communication with serial port
    def closeSerialComm(self):
        print('     Stop serial pooling!')
        SerialInterface.serialPort.close()      

try:
    serialInterface = SerialInterface()
except:
    serialInterface = SerialInterface()


### Read binary file and create 4-byte packets

In [2]:
# ********** Read binary file to be flashed **********
import math

BYTES_PER_PACKET = 4

#fileHandle = open("../STM32_FileSystem/Debug/STM32_FileSystem.bin", mode="rb")
fileHandle = open("../STM32_SnakeGameNokia/Debug/STM32_SnakeGameNokia.bin", mode="rb")
fileData = fileHandle.read()
fileSizeBytes = len(fileData)
fileSizePackets = math.ceil(len(fileData) / BYTES_PER_PACKET)

print("--> File Size: ", fileSizeBytes, " <--")
print("--> Nr. of packets: ", fileSizePackets, " <--")

# Create array of packets
filePackets = []
for idx in range(fileSizePackets):
    try:
        filePackets.append(fileData[idx * BYTES_PER_PACKET  :  (idx*BYTES_PER_PACKET) + BYTES_PER_PACKET])
    except:
        print("filePacket - Error trying to packet position ", idx*BYTES_PER_PACKET, " to ", (idx*BYTES_PER_PACKET) + BYTES_PER_PACKET, " Fallback to pack only the existing bytes.")
        filePackets.append(fileData[idx *BYTES_PER_PACKET  :  ])

print("--> Packets example: ", filePackets[0].hex() , " - ",filePackets[1].hex() , " - ",filePackets[2].hex() )

--> File Size:  74384  <--
--> Nr. of packets:  18596  <--
--> Packets example:  00000220  -  851e0408  -  ed170408


### Configure State Machine

In [3]:
# ********** Create State Machine **********
import numpy as np
# Super-class for state-machine implementation
class SuperState( object ):
    def someStatefulMethod( self, input ):
        pass
    def transitionRule( self, input ):
        raise NotImplementedError()
    
# States implementation
class Idle_01( SuperState ):
    def transitionRule( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x80:
            print("--> Device found! Starting communication to flash SW + sending SW info to MCU <--")
            
            # Send the answer command "C0" + number of bytes in current SW packet (ex.: 0xC0001234)
            fileSizePacketsHex =  fileSizePackets.to_bytes(3,'big')
            cmm = bytes(bytes({0xC0}) + fileSizePacketsHex)
            serialInterface.writeSerialBuffer(cmm.hex()) 
            return Prepare2Flash_02()
        
class Prepare2Flash_02( SuperState ):
    def someStatefulMethod( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x81:
            print("--> Erasing flash memory - Sector ", input[1] , " erased! <--")

    def transitionRule( self, input ):
        if input is not None and len(input) > 0 and input[0] == 0x82:
            print("--> Sw authorized to be flashed <--")
            return SendFlashData_03()
        
class SendFlashData_03( SuperState ):
    def __init__(self):
        self.packetNum = 0 # Counter of transmitted packets    
    
    def someStatefulMethod( self, input ):        
        if self.packetNum < fileSizePackets:
            serialInterface.writeSerialBuffer(filePackets[self.packetNum].hex(), echo=0) # Transmit binary packet (no echo)
            self.packetNum += 1 #Increment packet counter

        if np.remainder(self.packetNum, 100) == 0: # User feedback
            print("--> Transmitted packets: ", self.packetNum, " <--")
            
    def transitionRule( self, input ):
        #if self.packetNum == fileSizePackets and \
        if input is not None and len(input) > 0 and input[0] == 0x84: # End of Flashing - All packets transmitted + MCU returns 0x84
            print("--> Transmission finished succesfully - Transmitted ", self.packetNum, " of ", fileSizePackets, " <--")
            serialInterface.closeSerialComm()
            return Idle_01()
        elif input is not None and len(input) > 0 and input[0] == 0x8F: # Error - MCU returned error
            print("--> Error! <--")
            return Idle_01()
        
class Error_99( SuperState ):
    def someStatefulMethod( self, input ):
        pass
    def transitionRule( self, input ):
        pass

# Context class - the state-machine "engine"
class Context:
    def __init__(self, initial_state):
        self._state = initial_state

    def execute(self,input):
        self._state.someStatefulMethod(input)
        _next_state = self._state.transitionRule(input)
        if _next_state is not None:
            self._state = _next_state

# Declare instance of Context, starting in the initial state
context = Context(Idle_01())


### Main Program execution

In [4]:
import time
import keyboard # using module keyboard

# ********** Main Program Flow **********
print("--> SW Loader @ V1.0. Waiting for device communication <--")
print("--> Matheus Sozza @ 2023 <--")
attemptCntr = 0

while 1:
    serialString = serialInterface.readSerialBuffer()
    context.execute(serialString)
    if serialString == None:
        attemptCntr +=1
    time.sleep(0.010) #

    if keyboard.is_pressed('q') or attemptCntr >= 600000000000: # if key 'a' is pressed OR timeout
        print('Stop serial pooling!')
        serialInterface.closeSerialComm()
        break # finishing the loop

--> SW Loader @ V1.0. Waiting for device communication <--
--> Matheus Sozza @ 2023 <--
     RX->  80ffffff
--> Device found! Starting communication to flash SW + sending SW info to MCU <--
     TX->  c00048a4
     RX->  8106ffff
--> Erasing flash memory - Sector  6  erased! <--
     RX->  8107ffff
--> Erasing flash memory - Sector  7  erased! <--
     RX->  8108ffff
--> Erasing flash memory - Sector  8  erased! <--
     RX->  8109ffff
--> Erasing flash memory - Sector  9  erased! <--
     RX->  810affff
--> Erasing flash memory - Sector  10  erased! <--
     RX->  810bffff
--> Erasing flash memory - Sector  11  erased! <--
     RX->  820048a4
--> Sw authorized to be flashed <--
--> Transmitted packets:  100  <--
--> Transmitted packets:  200  <--
--> Transmitted packets:  300  <--
--> Transmitted packets:  400  <--
--> Transmitted packets:  500  <--
--> Transmitted packets:  600  <--
--> Transmitted packets:  700  <--
--> Transmitted packets:  800  <--
--> Transmitted packets:  900  <